<center><h1><b>Battle of neighbourhoods</b></h1>
<h2>Toronto edition</h2>
<h4>By <a href="franciscot.dev">Francisco Tarantuviez</a></h4></center>
 <hr />

### Problem to solve
    I am currently living in the West Side of Toronto, Canada in a neighbourhood called Dufferin, Dovercourt Village. I love where I live because all the amenities that the neighbourhood has (such as restaurants, gyms, coffee shops, etc). The problem start when I recive a spectacular job offer that I cannot reject, but the place of the job is far of Dufferin. So, I want to take a good decision, a data driven decision. Then, via Machine Learning techniques and Exploratory Data Analysis (EDA) I'll understand and compare the neighbourhoods in all Toronto having two principal parameters:

       1. Distance between the neighbourhood and the job place
   
       2. Similarity between my actual neighbourhood and the future one

### Dataset

The dataset already passed the ETL process (Extract, Transform and Load). Currently is loaded in a database in IBM storage, but you can find it in this [url](https://github.com/ftarantuviez/data_science_capstone/blob/master/project_toronto/ETL_notebooks/datasets/toronto-venues.csv)

Let's review the dataset to understand it a little more:

- Number of rows: 2378
- Number of columns: 11

##### Columns:
- ven_id (string): the venue id in Foursquare API
- ven_name (string): name of the venue
- ven_lat (float): latitude of the venue
- ven_lng (float): longitude of the venue
- ven_category_name (string): venue category name
- ven_category_id (string): identifier value of the category in Foursquare API
- ven_address (string): street and number of the venue
- neighbourhood (string): name of the neighbourhood
- neighbourhood_lat (float): neighbourhood latitude
- neighbourhood_lng (float): neighbourhood longitude
- borough (string): borough of the neighbourhood name

In [1]:
# packages that will be used

# basic data manipulation
import pandas as pd
import numpy as np

# visualation packages
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# database
import ibm_db
import ibm_db_dbi

# geospacial operations
import geopy.distance

# data transformation and modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN

In [2]:
# useful variables
current_neigh = "Dufferin, Dovercourt Village"
job_neigh = "Rouge Hill, Port Union, Highland Creek"

In [3]:
# magic sql
%load_ext sql

In [4]:
# open connection with the database
%sql ibm_db_sa://jts01551:67sk^vschz4h148z@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB

DB2/LINUXX8664


'Connected: jts01551@BLUDB'

In [5]:
%sql SELECT TABSCHEMA, TABNAME, CREATE_TIME FROM SYSCAT.TABLES where TABSCHEMA='JTS01551' ORDER BY create_time DESC

 * ibm_db_sa://jts01551:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,create_time
JTS01551,TORONTO-VENUES-NEIGHBOURHOODS,2021-01-31 16:30:41.041424
JTS01551,CHICAGO_CRIME,2021-01-23 20:54:42.959854
JTS01551,CHICAGO_SCHOOLS,2021-01-23 20:50:06.734306
JTS01551,CHICAGO_SOCIOECONOMIC,2021-01-23 20:47:04.596609
JTS01551,SCHOOLS,2021-01-23 19:12:19.655471
JTS01551,SCHOOLS_CHICAGO,2021-01-23 19:05:07.347107
JTS01551,MCDONALDS,2021-01-22 16:45:44.098727
JTS01551,INSTRUCTOR,2021-01-22 04:23:41.116508
JTS01551,PETRESCUE,2021-01-21 15:49:04.715313
JTS01551,LOCATIONS,2021-01-21 13:22:55.610166


In [6]:
result = %sql SELECT * FROM "TORONTO-VENUES-NEIGHBOURHOODS"

 * ibm_db_sa://jts01551:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


In [7]:
df = result.DataFrame()

In [8]:
df.head()

,ven_id,ven_name,ven_lat,ven_lng,ven_category_name,ven_category_id,ven_address,neighbourhood,neighbourhood_lat,neigbourhood_long,borough
0,4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,43.751976046055574,-79.33214044722958,Park,4bf58dd8d48988d163941735,Toronto,Parkwoods,43.75,-79.33,North York
1,4e13a489b0fb5dfdd0756c90,PetSmart,43.748639084843475,-79.33348827385937,Pet Store,4bf58dd8d48988d100951735,Ontario,Parkwoods,43.75,-79.33,North York
2,4cb11e2075ebb60cd1c4caad,Variety Store,43.75197441585782,-79.33311418516017,Food & Drink Shop,4bf58dd8d48988d1f9941735,29 Valley Woods Road,Parkwoods,43.75,-79.33,North York
3,4aec8a50f964a5207ac821e3,Armenian Kitchen,43.73107096414275,-79.30539020048727,Middle Eastern Restaurant,4bf58dd8d48988d115941735,1646 Victoria Park Ave,Victoria Village,43.73,-79.31,North York
4,4fff5c20e4b02bc662e83585,Blue Water Curry,43.731046,-79.30533000000000,Caribbean Restaurant,4bf58dd8d48988d144941735,1646 Victoria Park,Victoria Village,43.73,-79.31,North York


### Lets check the features

We can see some columns that are not useful for this notebook, such as the venue id in the foursquare API. In other moment, with that value we can extract more data, but now is not the idea.
So, let's remove the following columns:
- ven_id
- ven_category_id

In [9]:
df.drop(columns=["ven_id", "ven_category_id"], inplace=True)

### Add new features

We already have the location of each neighbourhood, but let get it the distance in kilometers in relation with the job neighbourhood.
We can find the distance in kilometers with help of the module *geopy* with the function distance

In [10]:
job_neigh_data = df[df["neighbourhood"] == job_neigh][["neighbourhood_lat", "neigbourhood_long"]].values[0]
job_neigh_loc = (job_neigh_data[0],job_neigh_data[1])
get_distance_job_neigh = lambda x: np.round(
        geopy.distance.distance(
            (x["neighbourhood_lat"], x["neigbourhood_long"]),job_neigh_loc).km ,2)

df["dist_job_neigh_km"] = df.apply(get_distance_job_neigh, axis=1)

df.head()

,ven_name,ven_lat,ven_lng,ven_category_name,ven_address,neighbourhood,neighbourhood_lat,neigbourhood_long,borough,dist_job_neigh_km
0,Brookbanks Park,43.751976046055574,-79.33214044722958,Park,Toronto,Parkwoods,43.75,-79.33,North York,14.39
1,PetSmart,43.748639084843475,-79.33348827385937,Pet Store,Ontario,Parkwoods,43.75,-79.33,North York,14.39
2,Variety Store,43.75197441585782,-79.33311418516017,Food & Drink Shop,29 Valley Woods Road,Parkwoods,43.75,-79.33,North York,14.39
3,Armenian Kitchen,43.73107096414275,-79.30539020048727,Middle Eastern Restaurant,1646 Victoria Park Ave,Victoria Village,43.73,-79.31,North York,13.80
4,Blue Water Curry,43.731046,-79.30533000000000,Caribbean Restaurant,1646 Victoria Park,Victoria Village,43.73,-79.31,North York,13.80


[//]: # "This space was intentionally left in blank :p"

Here below we can notice the distance between our current neighbourhood to the job neighbourhood

In [11]:
print("""The distance of {} (current neighbourhood) in relation with {} (job neighbourhood) is {}km
      """.format(current_neigh, job_neigh, df[df["neighbourhood"] == current_neigh]["dist_job_neigh_km"].values[0]))

The distance of Dufferin, Dovercourt Village (current neighbourhood) in relation with Rouge Hill, Port Union, Highland Creek (job neighbourhood) is 26.2km
      


In [12]:
df.groupby("neighbourhood").count()

,ven_name,ven_lat,ven_lng,ven_category_name,ven_address,neighbourhood_lat,neigbourhood_long,borough,dist_job_neigh_km
neighbourhood,,,,,,,,,
Agincourt,10,10,10,10,10,10,10,10,10
"Alderwood, Long Branch",7,7,7,7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",4,4,4,4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26,26,26,26
...,...,...,...,...,...,...,...,...,...
"Willowdale, Willowdale West",3,3,3,3,3,3,3,3,3
Woburn,3,3,3,3,3,3,3,3,3
Woodbine Heights,8,8,8,8,8,8,8,8,8


### One Hot Encoding

Let's extract useful information about the types of categories in each neighbourhood. To achive that, let's use One Hot Encoding technique

In [28]:
venues_onehot = pd.get_dummies(df[["ven_category_name"]], prefix="", prefix_sep="")
venues_onehot["neighourhood"] = df["neighbourhood"]

fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]
venues_onehot.head()

,neighourhood,Accessories Store,Airport,American Restaurant,Animal Shelter,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# function to normalize the names